In [1]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_FLEET_SIZE,
    PATH_RESULTS_SUMMARY,
    PATH_RESULTS_SINGLE_MODAL_BENCHMARK,
    PATH_RESULTS_VALUE_STOCHASTIC,
    PATH_RESULTS_VAR_REGION,
    PATH_RESULTS_VAR_TUPLE,
)
from modules.stochastic_program.factory import StochasticProgramFactory



# Prepare Model Parameters
## Scenario Data
Make scenarios complete cross product

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [5]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [6]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [7]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [8]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [9]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Other Model Parameters


In [10]:
distances = pd.read_pickle(PATH_DISTANCES)

In [11]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [12]:
real_fleet_size = pd.read_pickle(PATH_FLEET_SIZE).to_dict()['id']

# Create & Solve Stochastic Program

In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
vehicle_types = ALL_VEHICLE_TYPES
factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df, vehicle_types)
factory.set_initial_allocation(real_fleet_size)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.06 seconds
_convert_demand finished in 0.08 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.14 seconds
_set_max_demand finished in 0.02 seconds
set_initial_allocation finished in 0.00 seconds


In [15]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [16]:
stochastic_program.relocations_disabled = False

In [17]:
stochastic_program.create_model()

_create_variables finished in 0.79 seconds
_create_objective_function finished in 0.98 seconds
_create_demand_constraints finished in 1.14 seconds
_create_relocation_binary_constraints finished in 0.01 seconds
_create_big_u_sum_constraints finished in 0.15 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
_create_no_refused_demand_constraints finished in 0.01 seconds
_create_relocations_constraints finished in 0.41 seconds
_create_vehicle_trips_starting_constraints finished in 0.16 seconds
_create_vehicle_trips_ending_constraints finished in 0.18 seconds
_create_initial_allocation_constraints finished in 0.00 seconds
_create_non_anticipativity_constraints finished in 0.25 seconds
_create_constraints finished in 2.31 seconds
create_model finished in 4.74 seconds


In [18]:
stochastic_program.solve()

Status: Optimal
Optimal Value of Objective Function:  39439.17340320907
solve finished in 15.58 seconds


In [19]:
41680.24682238031
4.28

4.28

In [20]:
scenarios.reset_index().nunique()

scenarios         2
start_hex_ids    82
end_hex_ids      82
time              2
vehicle_types     3
demand           79
dtype: int64

In [21]:
stochastic_program.get_summary()

get_results_by_tuple_df finished in 0.67 seconds
get_results_by_region_df finished in 0.01 seconds
get_summary finished in 0.72 seconds


{'status': 'Optimal',
 'objective': 39439.17340320907,
 'n_trips_avg': 7387.5,
 'n_unfilled_demand_avg': 2210.0,
 'demand_avg': 9597.5,
 'n_parking_avg': 18146.5,
 'n_relocations_avg': 3688.0}

In [22]:
{'status': 'Optimal',
 'objective': 41680.24682238031,
 'n_trips_avg': 7586.0,
 'n_unfilled_demand_avg': 2168.0,
 'demand_avg': 9754.0,
 'n_parking_avg': 17948.0,
 'n_relocations_avg': 3540.0}

{'status': 'Optimal',
 'objective': 41680.24682238031,
 'n_trips_avg': 7586.0,
 'n_unfilled_demand_avg': 2168.0,
 'demand_avg': 9754.0,
 'n_parking_avg': 17948.0,
 'n_relocations_avg': 3540.0}

In [23]:
os.makedirs(os.path.dirname(PATH_RESULTS_VAR_REGION), exist_ok=True)
stochastic_program.get_results_by_region_df().to_pickle(PATH_RESULTS_VAR_REGION)
stochastic_program.get_results_by_tuple_df().to_pickle(PATH_RESULTS_VAR_TUPLE)

get_results_by_region_df finished in 0.02 seconds
get_results_by_tuple_df finished in 0.50 seconds


# Benchmarks
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information

In [24]:
capacities = [
    {
        "kick_scooter": 10000,
        "bicycle": 4000,
        "car": 3000,
    },
    {
        "kick_scooter": 8000,
        "bicycle": 3000,
        "car": 2000,
    },
    {
        "kick_scooter": 6000,
        "bicycle": 2000,
        "car": 1000,
    },
]


In [25]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.24 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.26 seconds
_set_max_demand finished in 0.02 seconds
Status: Optimal
Optimal Value of Objective Function:  41986.89074871598
solve finished in 24.41 seconds


Status: Optimal
Optimal Value of Objective Function:  42184.083245026224
solve finished in 14.60 seconds


Status: Optimal
Optimal Value of Objective Function:  32621.697044450702
solve finished in 5.67 seconds


Status: Optimal
Optimal Value of Objective Function:  32664.57866421464
solve finished in 5.44 seconds


Status: Optimal
Optimal Value of Objective Function:  39492.22701510469
solve finished in 18.24 seconds


Status: Optimal
Optimal Value of Objective Function:  39660.30488397726
solve finished in 9.34 seconds


Status: Optimal
Optimal Value of Objective Function:  29844.08240653139
solve finished in 5.45 seconds


S

In [26]:
results_df = pd.DataFrame.from_dict(results)

In [27]:
results_df

,status,objective,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,41986.890749,7810.0,1787.5,9597.5,26190.0,3491.0,10000,4000,3000,False,False
1,Optimal,42184.083245,7899.0,1698.5,9597.5,26101.0,3160.5,10000,4000,3000,False,True
2,Optimal,32621.697044,5212.0,4385.5,9597.5,28788.0,0.0,10000,4000,3000,True,False
3,Optimal,32664.578664,5215.0,4382.5,9597.5,28785.0,0.0,10000,4000,3000,True,True
4,Optimal,39492.227015,7406.0,2191.5,9597.5,18594.0,3703.0,8000,3000,2000,False,False
5,Optimal,39660.304884,7460.5,2137.0,9597.5,18539.5,3283.0,8000,3000,2000,False,True
6,Optimal,29844.082407,4446.5,5151.0,9597.5,21553.5,0.0,8000,3000,2000,True,False
7,Optimal,29861.529239,4443.5,5154.0,9597.5,21556.5,0.0,8000,3000,2000,True,True
8,Optimal,35886.382460,6810.5,2787.0,9597.5,11189.5,3907.0,6000,2000,1000,False,False
9,Optimal,36129.948334,6821.0,2776.5,9597.5,11179.0,3517.5,6000,2000,1000,False,True


In [28]:
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark

In [44]:
last_demand = scenarios.copy()

last_demand_unstacked = last_demand.unstack("vehicle_types")
last_demand_unstacked[("demand", "car")] = (
    last_demand_unstacked[("demand", "kick_scooter")]
    + last_demand_unstacked[("demand", "car")]
    + last_demand_unstacked[("demand", "bicycle")]
)
last_demand_unstacked
last_demand: pd.DataFrame = (
    last_demand_unstacked["demand"]["car"].to_frame().stack().to_frame(name="demand")
)
last_demand.index = last_demand.index.set_names(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)


In [45]:
results = []

for vehicle_types in [["car"], ["bicycle"], ["kick_scooter"]]:
    current_vehicle_type = vehicle_types[0]
    print(current_vehicle_type)
    current_fleet_capacity = {
        current_vehicle_type: real_fleet_size[current_vehicle_type]
    }

    last_demand.index = last_demand.index.set_levels(
        last_demand.index.get_level_values("vehicle_types").map(
            lambda x: current_vehicle_type
        ),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        last_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"accumulated_unfulfilled_demand": "demand"}
    )

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


car
Status: Optimal
Optimal Value of Objective Function:  17974.126381656068
solve finished in 2.21 seconds
bicycle
Status: Optimal
Optimal Value of Objective Function:  3367.3803592446247
solve finished in 2.11 seconds
kick_scooter
Status: Optimal
Optimal Value of Objective Function:  15843.97947509692
solve finished in 2.43 seconds


In [46]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(real_fleet_size)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


Status: Optimal
Optimal Value of Objective Function:  39439.17340320906
solve finished in 18.78 seconds


In [47]:
results = pd.DataFrame.from_dict(results)

In [48]:
# unfulfilled demand and demand not sinnvoll here
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,39439.173403,7387.5,2210.0,9597.5,18146.5,3704.0,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,37185.486216,7791.5,15692.0,23483.5,17742.5,4509.0,['car']['bicycle']['kick_scooter']


In [34]:
compare.to_pickle(PATH_RESULTS_SINGLE_MODAL_BENCHMARK)

## Value Of The Stochastic Solution

In [35]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [36]:
results = []
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


Status: Optimal
Optimal Value of Objective Function:  19223.537294747617
solve finished in 5.90 seconds
Status: Optimal
Optimal Value of Objective Function:  20386.912564257425
solve finished in 4.07 seconds


In [37]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,19223.537295,7298.0,2115.0,9413.0,18236.0,3325.0,"[kick_scooter, bicycle, car]"
1,Optimal,20386.912564,7606.0,2176.0,9782.0,17928.0,3545.0,"[kick_scooter, bicycle, car]"


In [38]:
results_df.to_pickle(PATH_RESULTS_VALUE_STOCHASTIC)